## SKFlow Simple Linear Regression on Parkinson's Disease Telemonitoring Dataset

https://archive.ics.uci.edu/ml/datasets/Parkinsons+Telemonitoring

In [82]:
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.learn.python.learn as skflow
from sklearn import datasets, metrics
import numpy as np
import os.path
import pickle

### Function Defintions

In [95]:
def convert_dataframe(dataframe, colnames, labelcol):
    ### Converting Data into Tensors
    X = np.array(dataframe[colnames].iloc[:,]).astype(np.float32)
    Y = np.array(dataframe[labelcol].iloc[:,]).astype(np.float32)
    ### Reduce Labels to discrete values (Categorical)
    #labels = list(set(df[labelcol]))
    #Y = np.array([labels.index(x) for x in df[labelcol]]).astype(np.float32)
    
    ### Number of label categories
    #labelcats = df[labelcol].unique().shape[0]+1    
    return X, Y #,labelcats

### load or generate random ids for test and training sets
def slice_ids(length,run_id):
    idfile = "models/"+run_id+"_ids"
    if os.path.isfile(idfile): 
        ### Re-run model
        with open(idfile, 'rb') as fp:
            randomInd = pickle.load(fp)
    else:
        ### First run
        randomInd = np.random.permutation(length)
        with open(idfile, 'wb') as fp:
            pickle.dump(randomInd, fp)
    
    ###  70% of ids randomly selected for training
    ###  30% of ids randomly selected for testing
    mid = int(.7*length)
    trainidx = randomInd[:mid]
    testidx = randomInd[mid:]
    return trainidx, testidx

def eval_classifier(classifier, testX, testY):
    ### Evaluate Model
    y_p=classifier.predict(testX)
    return np.linalg.norm(testY-y_p)/testY.shape[0]

def run_classifier(run_id, steps):
    ### Get Training and Testing indexes
    trainidx, testidx = slice_ids(len(X),run_id)

    ### Define classifier - Simple Linear Regression
    ### model_dir - this is where the model is saved. To re-run use the same runIdentifier
    model = skflow.LinearRegressor(
        feature_columns=feature_columns,
        model_dir="models/"+run_id,
        enable_centered_bias=False)
    
    model.fit(X[trainidx,:], 
              Y[trainidx],#.astype(np.int), 
              steps=steps)
    return eval_classifier(model,X[testidx,:],Y[testidx])

### Load data and convert to tensors

In [97]:
### Load datafile into dataframe
datafile = "data/data.txt"
df_data = pd.read_csv(datafile)

### Create list of column names for feature data.
featcol = list(df_data.columns.values)
featcol.remove("subject#")
featcol.remove("total_UPDRS")
featcol.remove("motor_UPDRS")

### First Model fitting Motor_UPDRS - SLR

In [ ]:
labelcol = "motor_UPDRS" 
### Convert data into tensors
X, Y = convert_dataframe(df_data, featcol, labelcol)
feature_columns = skflow.infer_real_valued_columns_from_input(X)
run_classifier("SLR_Motor_UPDRS_001", 10000)

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.


0.18479158657894038

### Second Model fitting Total_UPDRS - SLR

In [ ]:
labelcol = "total_UPDRS"
### Convert data into tensors
X, Y = convert_dataframe(df_data, featcol, labelcol)
feature_columns = skflow.infer_real_valued_columns_from_input(X)
run_classifier("SLR_Total_UPDRS_001", 10000)